# Data Cleaning and Visualisation

EDA process includes:

- Removing Duplicates
- Fixing Values
- Normalizing if needed
- Cleaning Outliers
- Looking a feature distribution
- Seperating into categorical and numerical columns
- Visualizing Data